# Template Matching

In [12]:
#One time imports
import cv2
import numpy as np

from IPython.display import clear_output
%pylab inline 
pylab.rcParams['figure.figsize'] = (16.0, 12.0)

import matplotlib.pyplot as plt

Populating the interactive namespace from numpy and matplotlib


## Template Matching Algorithm

In [103]:
#Fetches templates and returns them in a dict
def getTemplates():
    #TODO: Automate this to traverse a template directory
    tearPath = "train_images/tear/Tear_pure.png"
    bowPath  = "train_images/bow/Bow_pure.png"
    beltPath = "train_images/belt/Belt_pure.png"
    cloakPath = "train_images/Cloak/Cloak_pure.png"
    templatePathes = [bowPath, tearPath, beltPath, cloakPath]
    threshold = 0.8

    #Create a dict of arrays to store item with path and pixel values
    templates = dict()
    for path in templatePathes: 
        itemName = path.split('/')[-1].split('.')[0].split('_')[0]
        item = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
        templates[itemName] = [path, item]
    return templates
templates = getTemplates()
#print(templates)
for k, v in templates.items(): 
    print(k, v[0])

Bow train_images/bow/Bow_pure.png
Tear train_images/tear/Tear_pure.png
Belt train_images/belt/Belt_pure.png
Cloak train_images/Cloak/Cloak_pure.png


In [123]:
#Is the template in the frame?
#def inFrame(template, frame, threshold): 
frame = cv2.imread("screenshots/belt/Tear_cloak_spat_belt_rod-4.png")
#gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
#template = templates['Tear'][1]


def inFrame(template, frame, threshold = 0.8):
    result = cv2.matchTemplate(gray_frame, template, cv2.TM_CCOEFF_NORMED)
    loc = np.where(result >= threshold)
    #if the first element is empty, the obj hasn't been found
    return any(loc[0])

#return the number of items in the frame
def countInFrame(template, frame, threshold = 0.8): 
    result = cv2.matchTemplate(gray_frame, template, cv2.TM_CCOEFF_NORMED)
    loc = np.where(result >= threshold)
    if (any(loc[0])): 
        return len(loc)//2
    return 0
    
itemsInFrame = countInFrame(templates['Bow'][1], frame, threshold=0.8)

for i in range(0, itemsInFrame):
    print(i)

In [124]:
videoPath = "Videos/Data/Tear/Tear_cloak_spat_belt_rod.mp4"
vid = cv2.VideoCapture(videoPath)
threshold = 0.8
frame_no = 0 
try:
    while True:
        # Capture frame-by-frame
        ret, frame = vid.read()     
        if not ret:
            # Release the Video Device if ret is false
            vid.release()
            # Message to be displayed after releasing the device
            print("Released Video Resource")
            break
        #Match on a gray version of the frame
        gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        
        found_items = []
        for name, val in templates.items(): 
            item = val[1]
            #check if item is in frame
            itemsInFrame = countInFrame(item, frame, threshold)
            for i in range(0, itemsInFrame):
                found_items.append(name)
        print("In frame " + str(frame_no) + ", found " + str(found_items))
        
        clear_output(wait=True)
        frame_no += 1
except KeyboardInterrupt:
    # Release the Video Device
    vid.release()
    # Message to be displayed after releasing the device
    cv2.destroyAllWindows()
    print("Released Video Resource")

Released Video Resource


In [22]:
# Put the code in try-except statements
# Catch the keyboard exception and 
# release the camera device and 
# continue with the rest of code.
def findIt(templatePath, videoPath, threshold = 0.7):
    vid = cv2.VideoCapture(videoPath)
    template = cv2.imread(templatePath, cv2.IMREAD_GRAYSCALE)
    w, h = template.shape[::-1]
    try:
        while True:
            # Capture frame-by-frame
            ret, frame = vid.read()     
            if not ret:
                # Release the Video Device if ret is false
                vid.release()
                # Message to be displayed after releasing the device
                print("Released Video Resource")
                break
            #Match on a gray version of the frame
            gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            res = cv2.matchTemplate(gray_frame, template, cv2.TM_CCOEFF_NORMED)
            #Find where the match is based on a threshold 
            loc = np.where(res >= threshold)
            #Draw a box around the matched point(s)
            no_matches = 0
            for pt in zip(*loc[::-1]):
                no_matches += 1
                cv2.rectangle(frame, pt, (pt[0] + w, pt[1] + h), (0, 255, 0), 3)
            # Display the frame
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            title('Using Threshold = ' + str(threshold) + ': ' 
                    + str(no_matches) + ' matches')
            imshow(frame)
            #key = cv2.waitKey(1)
            show()
            # Display the frame until new frame is available
            clear_output(wait=True)
    except KeyboardInterrupt:
        # Release the Video Device
        vid.release()
        # Message to be displayed after releasing the device
        cv2.destroyAllWindows()
        print("Released Video Resource")

### Test: Trivial Case (Template is Exact Cropping of Target in Video)

In [43]:
tPath = "train_images/tear/Tear_pure.png"
vidPath = "Videos/Data/Tear/Tear_Kha.mp4"
findIt(tPath, vidPath, threshold=.9)

Released Video Resource


### Test: Itermediate Case (Target Orientation Changes)

In [25]:
tPath = "train_images/tear/Tear_pure.png"
vidPath = "Videos/Data/Tear/Tear_Braum.mp4"
findIt(tPath, vidPath, threshold = 0.9)

Released Video Resource


In [87]:
tPath = "train_images/bow/Bow_pure.png"
vidPath = "Videos/Data/Tear/Tear_Kha.mp4"
findIt(tPath, vidPath, threshold=.9)

Released Video Resource
